In [1]:
def CIFAR_Bootstrap_Exp(Num_Tasks,Num_Epochs):
    %matplotlib notebook
    import cntk
    cntk.device.try_set_default_device(cntk.device.gpu(0))
    import os
    os.environ['KERAS_BACKEND'] = 'cntk'
    import keras as keras
    from keras.models import Sequential
    from keras.layers import LSTM, Dense,BatchNormalization,Dropout,Flatten, Conv1D, Conv2D, MaxPool2D
    from keras.losses import binary_crossentropy, categorical_crossentropy
    from keras.metrics import categorical_accuracy
    from keras import regularizers,optimizers
    from keras.regularizers import l2
    import numpy as np
    import scipy.io
    import matplotlib.pyplot as plt
    import gzip
    from keras.layers.core import Lambda
    from scipy.integrate import trapz
    import seaborn as sns

    from keras.layers.core import Lambda
    from keras import backend as K    
    
    print(K.cntk_backend.dev.use_default_device())

    def mini_batches(InputSample,BatchSize):
        Index = np.array(range(InputSample.shape[0]),dtype=int)
        NumBatches = np.int(InputSample.shape[0]/BatchSize)
        Removed = np.array([],dtype=int)

        BatchInd =[]
        for BatchLoop in range(NumBatches):
            RemainIndex = np.delete(Index,Removed)
            SampleInd = np.random.choice(RemainIndex,size=BatchSize,replace=False)
            Removed = np.append(Removed,SampleInd,axis=0)

            BatchInd.append(SampleInd)
        RemainIndex = np.delete(Index,Removed)
        BatchInd.append(RemainIndex)

        return BatchInd,NumBatches

    def unpickle(file):
        import pickle
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
        return dict

    def one_hot(label_list):
        NumLabels = len(label_list)
        MaxLabel = max(label_list)
        one_hot_labels = np.zeros([NumLabels,MaxLabel+1],dtype=int)
        for i in range(NumLabels):
            one_hot_labels[i,label_list[i]-1] = 1

        return one_hot_labels

    def one_hot_reverse(one_hot_labels):
        labels = []
        for i in range(one_hot_labels.shape[0]):
            Sample = one_hot_labels[i,:]
            labels.append(Sample.argmax())

        return np.asarray(labels)

    InitStr = 'cifar-10-python\cifar-10-batches-py\data_batch_'
    File = InitStr + np.str(1)
    Batch = unpickle(File)
    FullInputs = Batch[b'data']
    FullTargets = one_hot(Batch[b'labels'])


    for i in  range(4):
        File = InitStr + np.str(i+2)
        Batch = unpickle(File)
        FullInputs = np.append(FullInputs,Batch[b'data'],axis=0)
        FullTargets = np.append(FullTargets,one_hot(Batch[b'labels']),axis=0)

    Conv = 1
    if Conv == 1:
        FullInputs = np.reshape(FullInputs,[FullInputs.shape[0],3,32,32]).transpose(0,2,3,1)

    FullInputs = FullInputs.copy(order='C')

    Validation_Cutoff = 0.75

    Validation_Cutoff = np.int(Validation_Cutoff*FullInputs.shape[0])
    if Conv == 1:
        ValInputs = FullInputs[Validation_Cutoff:,:,:,:]
    else:
        ValInputs = FullInputs[Validation_Cutoff:,:]
    ValTargets = FullTargets[Validation_Cutoff:,:]

    if Conv == 1:
        TrainInputs = FullInputs[0:Validation_Cutoff,:,:,:]
    else:
        TrainInputs = FullInputs[0:Validation_Cutoff,:]

    TrainTargets = FullTargets[0:Validation_Cutoff,:]


    data_dim = TrainInputs.shape[1]
    if Conv == 1:
        data_dim2 = TrainInputs.shape[2]
        NumChannels = 3

    NumSamples = TrainInputs.shape[0]
    Num_Targets = TrainTargets.shape[-1]

    index = np.linspace(0,NumSamples,NumSamples,endpoint=False,dtype=int)

    reg_coeff = 0

    def Gen_Conv_Model(reg_coeff):
        model = Sequential()
        model.add(Conv2D(50,(3,3),activation='relu',input_shape =(data_dim,data_dim2,NumChannels),data_format="channels_last"))
        model.add(MaxPool2D())
        model.add(BatchNormalization())
        model.add(Conv2D(50,(3,3),activation='relu',data_format="channels_last"))
        model.add(MaxPool2D())
        model.add(BatchNormalization())
        model.add(Flatten())
        model.add(Dropout(0.25))
        model.add(Dense(100,activation='relu',kernel_regularizer=l2(reg_coeff)))
        model.add(Dropout(0.25))
    #     model.add(Dense(100,activation='relu',kernel_regularizer=l2(reg_coeff)))
        model.add(Dense(Num_Targets,activation = 'softmax',kernel_regularizer=l2(reg_coeff),input_shape =(data_dim,)))
        optim = optimizers.adam(lr=0.00001)
        model.compile(loss='categorical_crossentropy',optimizer=optim,metrics=['categorical_accuracy'])

        return model

    def Gen_FF_Model(reg_coeff):
        model = Sequential()
        model.add(Dense(300,activation='relu',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Dense(300,activation='relu',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Dense(300,activation='relu',input_shape =(data_dim,),kernel_regularizer=l2(reg_coeff)))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        model.add(Dense(Num_Targets,activation = 'softmax',kernel_regularizer=l2(reg_coeff),input_shape =(data_dim,)))
        optim = optimizers.adagrad(lr=0.00001,decay=0.95)
        model.compile(loss='categorical_crossentropy',optimizer=optim,metrics=['categorical_accuracy'])
        return model

    if Conv == 1:
        Uni_model = Gen_Conv_Model(reg_coeff)
    else:
        Uni_model = Gen_FF_Model(reg_coeff)

    Init_Weights = Uni_model.get_weights()

    def acquisition_function_BALD(model,samples,Num_Targets,temperature=1,Target_Ratio = 5):
        nb_MC_samples = 30
        MC_output = K.function([model.layers[0].input, K.learning_phase()], [model.layers[-1].output])
        MC_samples = np.zeros([nb_MC_samples,samples.shape[0],Num_Targets])
        learning_phase = True 
        for i in range(nb_MC_samples):
            MC_samples[i,:,:] = np.array([MC_output([samples, learning_phase])[0]])

        expected_entropy = - np.mean(np.sum(MC_samples * np.log(MC_samples + 1e-10), axis=-1), axis=0)  # [batch size]
        expected_p = np.mean(MC_samples, axis=0)
        entropy_expected_p = - np.sum(expected_p * np.log(expected_p + 1e-10), axis=-1)  # [batch size]
        BALD_acq = entropy_expected_p - expected_entropy

        Exp_BALD = np.exp(BALD_acq/temperature)
        Sampling_Prob = Exp_BALD/np.sum(Exp_BALD).astype(float)

        Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
        Target_Ratio = Target_Ratio
        if Max_Prob_Ratio < Target_Ratio:
            while Max_Prob_Ratio <Target_Ratio:
                temperature = temperature*0.99
                Exp_BALD = np.exp(BALD_acq/temperature)
                StoreSampling_Prob = Sampling_Prob.copy()
                Sampling_Prob = Exp_BALD/np.sum(Exp_BALD).astype(float)
                Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
                if np.isnan(Max_Prob_Ratio):
                    Sampling_Prob = StoreSampling_Prob.copy()
        else:
            while Max_Prob_Ratio > Target_Ratio:
                temperature = temperature*1.01
                Exp_BALD = np.exp(BALD_acq/temperature)
                StoreSampling_Prob = Sampling_Prob.copy()
                Sampling_Prob = Exp_BALD/np.sum(Exp_BALD).astype(float)
                Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
                if np.isnan(Max_Prob_Ratio):
                    Sampling_Prob = StoreSampling_Prob.copy()
        return Sampling_Prob


    def acquisition_function_dist_to_threshold(model,samples,Num_Targets,temperature=1,Target_Ratio=5):
        Output = model.predict(samples)
        Output -= 1/float(Num_Targets)
        Dist_to_Threshold = np.sum(np.abs(Output),1)
        Exp_Dist_to_Threshold = np.exp(Dist_to_Threshold/temperature)
        Exp_Dist_to_Threshold *= 1
    #     Exp_Dist_to_Threshold = Dist_to_Threshold
        Sampling_Prob = Exp_Dist_to_Threshold/np.sum(Exp_Dist_to_Threshold).astype(float)
        Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
        Target_Ratio = Target_Ratio
        if Max_Prob_Ratio < Target_Ratio:
            while Max_Prob_Ratio <Target_Ratio:
                temperature = temperature*0.99
                Exp_Dist_to_Threshold = np.exp(Dist_to_Threshold/temperature)
                StoreSampling_Prob = Sampling_Prob.copy()
                Sampling_Prob = Exp_Dist_to_Threshold/np.sum(Exp_Dist_to_Threshold).astype(float)
                Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
                if np.isnan(Max_Prob_Ratio):
                    Sampling_Prob = StoreSampling_Prob.copy()
        else:
            while Max_Prob_Ratio > Target_Ratio:
                temperature = temperature*1.01
                Exp_Dist_to_Threshold = np.exp(Dist_to_Threshold/temperature)
                StoreSampling_Prob = Sampling_Prob.copy()
                Sampling_Prob = Exp_Dist_to_Threshold/np.sum(Exp_Dist_to_Threshold).astype(float)
                Max_Prob_Ratio = Sampling_Prob.max()/Sampling_Prob.min()
                if np.isnan(Max_Prob_Ratio):
                    Sampling_Prob = StoreSampling_Prob.copy()
        return Sampling_Prob


    count = 0 
    Num_Epochs = Num_Epochs
    Num_BurnIn = 1
    Batch_Size = 100

    NumTasks = 3
    try:
        SwitchPoint = np.int(Num_Epochs/NumTasks)
        print(SwitchPoint)
    except:
        SwitchPoint = 1
    Smoothing_Constant = 0

    Val_Error = np.zeros([Num_Epochs,4])
    Val_Acc = np.zeros([Num_Epochs,4])

    Uni_model.fit(TrainInputs,TrainTargets,batch_size = Batch_Size,epochs=Num_Epochs,verbose=1,validation_data=[ValInputs,ValTargets])

    TestStr = 'cifar-10-python\cifar-10-batches-py\test_batch'

    TestInputs = Batch[b'data']
    TestTargets = one_hot(Batch[b'labels'])

    if Conv == 1:
        TestInputs = np.reshape(TestInputs,[TestInputs.shape[0],3,32,32]).transpose(0,2,3,1)

    Predicted_Classes = Uni_model.predict_classes(TestInputs)
    Test_Labels = one_hot_reverse(TestTargets)
    Diff = Predicted_Classes - Test_Labels
    Correct = Diff[Diff ==0 ]
    Uni_TestError = Correct.shape[0]/Test_Labels.shape[0]

    Dist_to_Threshold = acquisition_function_dist_to_threshold(Uni_model,TrainInputs,Num_Targets,1,10)

    Easy_Dist_to_Threshold_ArgSort = np.flipud(Dist_to_Threshold.argsort())
    Hard_Dist_to_Threshold_ArgSort = Dist_to_Threshold.argsort()

    D2T_Easy_Model = Gen_Conv_Model(reg_coeff)
    D2T_Easy_Model.set_weights(Init_Weights)
    del Uni_model

    NumTasks = Num_Tasks
    TaskSplitPoints = np.int(np.ceil(NumSamples/NumTasks))
    print(TaskSplitPoints)
    for i in range(NumTasks):
        Num_Epochs_Task = Num_Epochs*(1/(i+1))
        TaskInd = Easy_Dist_to_Threshold_ArgSort[0:np.min([(i+1)*TaskSplitPoints,NumSamples])]
        D2T_Easy_Model.fit(TrainInputs[TaskInd,:],TrainTargets[TaskInd],batch_size=Batch_Size,epochs = np.int(Num_Epochs_Task))

    Predicted_Classes = D2T_Easy_Model.predict_classes(TestInputs)
    Test_Labels = one_hot_reverse(TestTargets)
    Diff = Predicted_Classes - Test_Labels
    Correct = Diff[Diff ==0 ]
    D2T_Easy_TestError = Correct.shape[0]/Test_Labels.shape[0]

    D2T_Hard_Model = Gen_Conv_Model(reg_coeff)
    D2T_Hard_Model.set_weights(Init_Weights)
    del D2T_Easy_Model

    NumTasks = Num_Tasks
    TaskSplitPoints = np.int(np.ceil(NumSamples/NumTasks))
    print(TaskSplitPoints)
    for i in range(NumTasks):
        Num_Epochs_Task = Num_Epochs*(1/(i+1))
        TaskInd = Hard_Dist_to_Threshold_ArgSort[0:np.min([(i+1)*TaskSplitPoints,NumSamples])]
        D2T_Hard_Model.fit(TrainInputs[TaskInd,:],TrainTargets[TaskInd],batch_size=Batch_Size,epochs = np.int(Num_Epochs_Task))


    Predicted_Classes = D2T_Hard_Model.predict_classes(TestInputs)
    Test_Labels = one_hot_reverse(TestTargets)
    Diff = Predicted_Classes - Test_Labels
    Correct = Diff[Diff ==0 ]
    D2T_Hard_TestError = Correct.shape[0]/Test_Labels.shape[0]

    del D2T_Hard_Model
    
    Errors = np.array([Uni_TestError,D2T_Hard_TestError,D2T_Easy_TestError])

    return Errors

In [ ]:
import numpy as np
NumTests = 100
from IPython.display import clear_output

TestError_Record = np.zeros([3,NumTests])

for i in range(NumTests):
    clear_output()
    print('Run:')
    print(i)
    TestError_Record[:,i] = CIFAR_Bootstrap_Exp(5,50)
    np.save('CIFAR_Bootstrap_Perf_5_lr0.00001',TestError_Record)

Run:
0


Using CNTK backend


GPU[0] Tesla K40c
16
Train on 37500 samples, validate on 12500 samples
Epoch 1/50
37500/37500 [==============================] - 18s 487us/step - loss: 2.8826 - categorical_accuracy: 0.1438 - val_loss: 2.1963 - val_categorical_accuracy: 0.2381
Epoch 2/50
37500/37500 [==============================] - 18s 467us/step - loss: 2.4661 - categorical_accuracy: 0.2197 - val_loss: 2.0293 - val_categorical_accuracy: 0.2953
Epoch 3/50
37500/37500 [==============================] - 18s 468us/step - loss: 2.3036 - categorical_accuracy: 0.2556 - val_loss: 1.9222 - val_categorical_accuracy: 0.3316
Epoch 4/50
37500/37500 [==============================] - 18s 468us/step - loss: 2.1802 - categorical_accuracy: 0.2845 - val_loss: 1.8526 - val_categorical_accuracy: 0.3538
Epoch 5/50
37500/37500 [==============================] - 18s 467us/step - loss: 2.1067 - categorical_accuracy: 0.3038 - val_loss: 1.7935 - val_categorical_accuracy: 0.3716
Epoch 6/50
37500/37500 [==============================] - 18s 46

In [3]:
i

2